# Tokens, Devices E Memória De Conversa

Este notebook demonstra, dentro do contexto de LLMs:

*   Como ocorre a geração dos próximos tokens de um prompt;
*   Quais as diferenças entre devices
*   Como as LLMs lembram de conversas

## Sumário

1. Checar existência de GPU
2. Automatizar a detecção de GPU
3. Carregar e checar informações da LLM da biblioteca transformers
4. Testar o processamento da LLM
5.

---


# 1. Checar existência de GPU

Ao carregar um LLM da biblioteca transformers, precisamos informar se a máquina possui CPU ou GPU.

Utilizando o comando "nvidia-smi" é possível checar a existência de GPU na máquina - caso não haja, o comando não funcionará. Caso haja GPU, é possível checar outras informações sobre a performance da máquina.

Observação: O comando "nvidia-smi" é executado na command line do sistema, NÃO no python. No Google Colab, é necessário colocar um ponto de exclamação ("!") na frente de comandos a serem executados na cmd.

In [1]:
!nvidia-smi

Mon Sep 16 22:13:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

---


# 2. Automatizar a detecção de GPU

Outra maneira de detectar se a máquina possui GPU é utilizando a biblioteca torch, através da função "cuda.is_available()".

Podemos fazer um if para atribuir o valor de 'cuda', caso a máquina possua GPU, ou 'cpu', caso a máquina possua CPU, a uma variável. Dessa forma, podemos automatizar a definição do tipo de máquina utilizada ao carregar a LLM.

In [2]:
# Fazendo os imports necessários:
import time
import numpy as np
import torch
import transformers

# Vendo se a máquina possui GPU
print(f"GPU presente: {torch.cuda.is_available()}") # retorna True se encontrou uma GPU

# Atribuindo o valor do tipo de dispositivo a variável device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

GPU presente: True
cuda


---


# 3. Carregar e checar informações da LLM da biblioteca transformers

Carregamos nosso modelo utilizando o "pipeline" da biblioteca transformers. O "pipeline" serve para agrupar todas as funções necessárias ao processar a entrada e gerar a saída do prompt. Utilizaremos apenas 3 parâmetros: task (a função que o modelo executa); model (nome do modelo carregado); device (tipo de máquina utilizada, podendo ser CPU='cpu' ou GPU='cuda'). Utilizamos a variável device, definida anteriormente, para indicar o tipo de máquina utilizada.

Após carregar o modelo, trouxemos mais detalhes sobre seu funcionamento através do método "model".

O modelo carregado foi o TinyLlama ('TinyLlama/TinyLlama-1.1B-Chat-v1.0'), um modelo textual generativo básico.

In [3]:
# Carregando o modelo
tiny_llama = transformers.pipeline(
    task='text-generation',
    model='TinyLlama/TinyLlama-1.1B-Chat-v1.0',
    device=device
)

# Exibindo informações do modelo carregado
print(tiny_llama)
print(tiny_llama.model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): 

---


# 4. Testar o processamento da LLM


Para testar o modelo, informamos um pequeno prompt, modificando apenas o parâmetro "max_new_tokens". Esse parâmetro informa a LLM a quantidade máxima de tokens que serão gerados. Caso ele não seja fornecido, o modelo irá gerar até encontrar um token que indique fim de sentença.

In [4]:
# Prompt que será processado pela LLM
prompt = "Today is a beautiful day to"

# Utilizando LLM para processar o prompt. Número máximo de novos tokens setado para 25.
output = tiny_llama(prompt, max_new_tokens=25)

# Printando o resultado
print(output)
print(150*'-')
print(output[0]['generated_text'])

[{'generated_text': 'Today is a beautiful day to be alive.\n\nVerse 2:\nThe sun is shining bright, the birds are singing,\nThe'}]
------------------------------------------------------------------------------------------------------------------------------------------------------
Today is a beautiful day to be alive.

Verse 2:
The sun is shining bright, the birds are singing,
The


Utilizando o tokenizer do GPT (https://platform.openai.com/tokenizer), podemos conferir como o GPT separa o resultado do nosso teste em tokens. Assim, como nosso modelo, o GPT separou em 25 tokens.

---


# 5. Tokenizar o prompt

Utilizamos o "tokenizer.encode" para transformar o prompt inserido anteriormente em tokens.

Tensores são estruturas (arrays) utilizadas para representar dados dentro de redes neurais. Por meio do parâmetro "return_tensors", informamos o tipo de tensor que será usado. Podemos utilizar tensores 'pt' (formato pytorch) ou 'tf' (formato tensorflow).

O método "to(device)" é usado apenas com tensores. Ele serve para mover o array para o tipo de dispositivo indicado (GPU='cuda' ou CPU='cpu').

In [5]:
# Transformando o prompt em tokens com o tensor 'pt'
prompt_input_ids = tiny_llama.tokenizer.encode(prompt, return_tensors='pt').to(device)

# Printando o prompt em formato de tokens
print(prompt_input_ids)

tensor([[    1, 20628,   338,   263,  9560,  2462,   304]], device='cuda:0')


---


# 6. Inspecionar geração de tokens

Para inspecionar a geração de tokens, geramos um novo output usando o método "generate" e 2 novos parâmetros ("return_dict_in_generate" e "output_scores").

Quando setado para True, "return_dict_in_generate" irá retornar um dicionário com as informações da geração de tokens.

Quando setado para True, "output_scores" irá retornar as pontuações de cada token gerado.

Observação: Dessa vez, usamos o prompt já tokenizado, criado no tópico anterior.

In [6]:
# Criando um novo output, retornando informações da geração e pontuação de tokens
output = tiny_llama.model.generate(
    prompt_input_ids,
    max_new_tokens=25,
    return_dict_in_generate=True,
    output_scores=True,
)

# Printando as chaves de informações do output
print(output.keys())

odict_keys(['sequences', 'scores', 'past_key_values'])


---


# 7. Pegar transition scores

Transition scores são as probabilidades de cada palavra aparecer na sequência do prompt. Essa probabilidade é usada para gerar as sentenças respondidas.

Para pegar os transition scores, utilizamos o método "compute_transition_scores", informando os valores de "sequences" e "scores" encontrados no tópico anterior.

In [7]:
# Pegando os transition scores
transitions = tiny_llama.model.compute_transition_scores(
    output.sequences, output.scores, normalize_logits=True
)

# Printando os transition scores
print(transitions)

tensor([[-1.5248e+00, -5.8350e-01, -9.5887e-01, -1.5709e+00, -1.4556e+00,
         -1.5400e+00, -9.8371e-04, -2.2910e-02, -1.0781e+00, -6.2063e-02,
         -1.6614e-01, -2.1465e+00, -6.8962e-01, -9.2523e-01, -2.5270e-01,
         -4.2579e-03, -7.1955e-01, -2.9930e-01, -1.0642e+00, -1.2073e+00,
         -3.4697e-02, -1.2240e-01, -4.7705e-01, -1.8254e-01, -5.4561e-01]],
       device='cuda:0')


---


# 8. Pegar transition scores APENAS do que foi gerado

Quando geramos uma resposta, o prompt original retorna junto. Não precisamos da probabilidade dos tokens do prompt original, então os tiramos do nosso resultado.

Fizemos uma tabela contemplando o o id, o score, a string e a probabilidade de cada token para análise.

In [12]:
# Pegando tamanho do prompt original
tamanho_prompt = len(prompt_input_ids[0])
print(f"Tamanho do prompt original: {tamanho_prompt}")

# Verificando se pegamos o tamanho certo do prompt
print(f"Prompt retirado da resposta do modelo: {output.sequences[0][:tamanho_prompt]}")

# Pegando apenas o conteúdo que foi gerado
generated_tokens = output.sequences[0][tamanho_prompt:]
print(f"Resposta do modelo SEM o prompt: {generated_tokens}")

# Printando as pontuações de cada token gerado
print('|----------+--------+-----------+--------|')
print('| token id | score  | token str | prob % |')
print('|----------+--------+-----------+--------|')
for (token, score) in zip(generated_tokens, transitions[0]):
    if tiny_llama.tokenizer.decode(token) == '\n':
      continue
    print(f"| {token:8d} | {score.to('cpu').numpy():.3f} | {tiny_llama.tokenizer.decode(token):9s} | {np.exp(score.to('cpu').numpy()):.4f} |")
print('|----------+--------+-----------+--------|')

Tamanho do prompt original: 7
Prompt retirado da resposta do modelo: tensor([    1, 20628,   338,   263,  9560,  2462,   304], device='cuda:0')
Resposta do modelo SEM o prompt: tensor([  367, 18758, 29889,    13,    13,  6565,   344, 29871, 29906, 29901,
           13,  1576,  6575,   338,   528,  2827, 11785, 29892,   278, 17952,
          526, 23623, 29892,    13,  1576], device='cuda:0')
|----------+--------+-----------+--------|
| token id | score  | token str | prob % |
|----------+--------+-----------+--------|
|      367 | -1.525 | be        | 0.2177 |
|    18758 | -0.583 | alive     | 0.5579 |
|    29889 | -0.959 | .         | 0.3833 |
|     6565 | -1.540 | Ver       | 0.2144 |
|      344 | -0.001 | se        | 0.9990 |
|    29871 | -0.023 |           | 0.9774 |
|    29906 | -1.078 | 2         | 0.3402 |
|    29901 | -0.062 | :         | 0.9398 |
|     1576 | -2.146 | The       | 0.1169 |
|     6575 | -0.690 | sun       | 0.5018 |
|      338 | -0.925 | is        | 0.3964 |
|   

---


# 9. Calcular tempo de execução

Utilizando o prompt inicial, fizemos um teste básico para calcular o tempo que nossa máquina leva para gerar uma resposta de, no máximo, 25 tokens (podemos modificar o valor para fazer outros experimentos).

In [14]:
# Pegando o tempo inicial
start = time.time()

# Gerando a resposta
output = tiny_llama(prompt, max_new_tokens=100)

# Pegando o tempo final
end = time.time()

# Printando o resultado
print(f'Tempo de execucao: {end - start:.2f} segundos')
print(output)
print(150*'-')
print(output[0]['generated_text'])

Tempo de execucao: 5.27 segundos
[{'generated_text': "Today is a beautiful day to be alive.\n\nVerse 2:\nThe sun is shining bright, the birds are singing,\nThe world is full of love and joy,\nLet's embrace this moment, let's be free,\nLet's live life to the fullest, let's be happy.\n\nChorus:\nToday, today, today, today, today, today, today, today, today, today, today, today, today, today"}]
------------------------------------------------------------------------------------------------------------------------------------------------------
Today is a beautiful day to be alive.

Verse 2:
The sun is shining bright, the birds are singing,
The world is full of love and joy,
Let's embrace this moment, let's be free,
Let's live life to the fullest, let's be happy.

Chorus:
Today, today, today, today, today, today, today, today, today, today, today, today, today, today


---


# 10. Lembrar da conversa

Se tentarmos conversar com o modelo sobre algo que foi dito anteriormente, ele não lembrará.

In [15]:
# Criando 2 prompts que se completam
prompt1 = "What day is today?"
prompt2 = "What day is tomorrow?"

# Gerando primeiro prompt
output = tiny_llama(prompt1, max_new_tokens=10)
print(output[0]['generated_text'])

print("-"*80)

# Gerando segundo prompt
output = tiny_llama(prompt2, max_new_tokens=10)
print(output[0]['generated_text'])

What day is today?

Student: It's Monday.

--------------------------------------------------------------------------------
What day is tomorrow?

Tomorrow is Monday.

Inter


Para fazer o modelo lembrar da conversa, precisamos alimentá-lo com TUDO que foi gerado anteriormente. Dessa forma, quando mandamos o segundo prompt, o modelo está recebendo a resposta do primeiro prompt E o segundo prompt.

In [16]:
# Criando 2 prompts que se completam
prompt1 = "What day is today?"
prompt2 = "What day is tomorrow?"

# Gerando primeiro prompt
output = tiny_llama(prompt1, max_new_tokens=10)
print(output[0]['generated_text'])

print("-"*80)

# Gerando segundo prompt, alimentando o modelo com a respota do primeiro prompt E o segundo prompt
output = tiny_llama(output[0]['generated_text'] + ". " + prompt2, max_new_tokens=30)
print(output[0]['generated_text'])

What day is today?

Student: It's Monday.

--------------------------------------------------------------------------------
What day is today?

Student: It's Monday.
. What day is tomorrow?

Student: It's Tuesday.
. What day is next week?

Student: It's Wednesday.


---


# 12. Apagar a LLM atual e carregar uma maior

Para testar um novo modelo, devemos primeiro apagar o modelo que já foi carregado e a memória que foi ocupada por ele.

Depois de apagar, vamos carregar o modelo "tiiuae/falcon-7b" (https://huggingface.co/tiiuae/falcon-7b). Ao rodar o código, vamos explodir a memória RAM (a máquina não tem capacidade de rodar a nova LLM).

In [ ]:
# Importando a biblioteca gc (garbage colector)
import gc

# Apagando a variável "tiny_llama"
del tiny_llama
# Liberando a memória de objetos que não estão sendo mais utilizados
gc.collect()
# Liberando a memória da GPU que está sendo utilizada por tensores
torch.cuda.empty_cache()

# Carregando o novo modelo "tiiuae/falcon-7b"
falcon = transformers.pipeline(
    task='text-generation',
    model='tiiuae/falcon-7b',
    device=device
)

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/16.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]